In [6]:
!pip install ultralytics

     -------------------------------------- 873.1/873.1 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 38.8/38.8 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 7.8/7.8 MB 6.5 MB/s eta 0:00:00
     -------------------------------------- 199.4/199.4 MB 5.8 MB/s eta 0:00:00
     -------------------------------------- 294.9/294.9 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 55.9/55.9 kB 2.9 MB/s eta 0:00:00
     ------------------------------------- 216.0/216.0 kB 12.9 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 10.8 MB/s eta 0:00:00
     -------------------------------------- 104.1/104.1 kB 6.3 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 9.9 MB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 MB 11.0 MB/s eta 0:00:00
     ------------------------------------- 536.2

In [16]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

# Загрузка модели YOLOv8
model = YOLO('yolov8n.pt')

# Список цветов для различных классов
colors = [
    (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255),
    (255, 0, 255), (192, 192, 192), (128, 128, 128), (128, 0, 0), (128, 128, 0),
    (0, 128, 0), (128, 0, 128), (0, 128, 128), (0, 0, 128), (72, 61, 139),
    (47, 79, 79), (47, 79, 47), (0, 206, 209), (148, 0, 211), (255, 20, 147)
]

# Функция для обработки изображения
def process_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)
    results = model(image)[0]
    
    # Получение оригинального изображения и результатов
    image = results.orig_img
    classes_names = results.names
    classes = results.boxes.cls.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy().astype(np.int32)

    # Подготовка словаря для группировки результатов по классам
    grouped_objects = {}

    # Рисование рамок и группировка результатов
    for class_id, box in zip(classes, boxes):
        class_name = classes_names[int(class_id)]
        color = colors[int(class_id) % len(colors)]  # Выбор цвета для класса
        if class_name not in grouped_objects:
            grouped_objects[class_name] = []
        grouped_objects[class_name].append(box)

        # Рисование рамок на изображении
        x1, y1, x2, y2 = box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Сохранение измененного изображения
    new_image_path = os.path.splitext(image_path)[0] + '_yolo' + os.path.splitext(image_path)[1]
    cv2.imwrite(new_image_path, image)

    # Сохранение данных в текстовый файл
    text_file_path = os.path.splitext(image_path)[0] + '_data.txt'
    with open(text_file_path, 'w') as f:
        for class_name, details in grouped_objects.items():
            f.write(f"{class_name}:\n")
            for detail in details:
                f.write(f"Coordinates: ({detail[0]}, {detail[1]}, {detail[2]}, {detail[3]})\n")

    print(f"Processed {image_path}:")
    print(f"Saved bounding-box image to {new_image_path}")
    print(f"Saved data to {text_file_path}")


for i in range(1,62): 
    process_image(f'./img/photo_{i}_2024-09-18_19-02-53.jpg')


0: 480x640 1 cat, 185.4ms
Speed: 0.0ms preprocess, 185.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Processed ./img/photo_1_2024-09-18_19-02-53.jpg:
Saved bounding-box image to ./img/photo_1_2024-09-18_19-02-53_yolo.jpg
Saved data to ./img/photo_1_2024-09-18_19-02-53_data.txt

0: 480x640 (no detections), 177.6ms
Speed: 8.0ms preprocess, 177.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Processed ./img/photo_2_2024-09-18_19-02-53.jpg:
Saved bounding-box image to ./img/photo_2_2024-09-18_19-02-53_yolo.jpg
Saved data to ./img/photo_2_2024-09-18_19-02-53_data.txt

0: 480x640 1 cat, 1 bear, 187.5ms
Speed: 8.0ms preprocess, 187.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Processed ./img/photo_3_2024-09-18_19-02-53.jpg:
Saved bounding-box image to ./img/photo_3_2024-09-18_19-02-53_yolo.jpg
Saved data to ./img/photo_3_2024-09-18_19-02-53_data.txt

0: 480x640 1 dog, 200.0ms
Speed: 0.0ms preprocess, 200.0ms inference, 0.0ms